# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of a *C++* library.

First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())

On Windows OSes, the visual studio version used to compile future wrappers must be given.

In [2]:
if is_windows:
    kwargs = dict(msvc_version = '12.0')
else:
    kwargs = dict()

* to detect the version of *Python* installed and to save it in the `PYTHON_VERSION` environment variable.

In [3]:
import sys
PYTHON_VERSION = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

* to import **AutoWIG**.

In [4]:
import autowig

* to import **subprocess**.

In [5]:
import subprocess

* to detect the **Git** repository root

In [6]:
import os
GIT_ROOT = subprocess.check_output('git rev-parse --show-toplevel', shell=True).decode()
GIT_ROOT = GIT_ROOT.replace('/', os.sep).strip()

Then, we need to install and compile the *C++* library.
To do so, we use available **Conda** recipes.

In [7]:
subprocess.call('conda remove -y libbasic', shell=True)
CONDA_RECIPE = os.path.join(GIT_ROOT, 'bin', 'conda', 'libbasic')
subprocess.check_call('conda build --python=' + PYTHON_VERSION + ' ' + CONDA_RECIPE + ' -c statiskit -c defaults --override-channels',
                      shell=True)
subprocess.check_call('conda install -y libbasic --use-local -c statiskit -c defaults --override-channels',
                      shell=True)

CalledProcessError: Command 'conda build --python=3.7 /home/pfernique/Desktop/develop/M2P2/FP17/bin/conda/libbasic -c statiskit -c defaults --override-channels' returned non-zero exit status 1.

Once these preliminaries are done, we can proceed to the actual generation of wrappers for this *C++* library.
For this, we create an empty Abstract Semantic Graph (ASG).

In [ ]:
asg = autowig.AbstractSemanticGraph()

We then parse the headers of this *C++* library with relevant compilation flags.

In [ ]:
%%time

try:
    from path import path as Path
except:
    from path import Path
prefix = Path(sys.prefix).abspath()
if is_windows:
    headers = [prefix/'Library'/'include'/'basic'/'overload.h',
               prefix/'Library'/'include'/'basic'/'binomial.h']
else:
    headers = [prefix/'include'/'basic'/'overload.h',
               prefix/'include'/'basic'/'binomial.h']
    
flags = ['-x', 'c++', '-std=c++11']
if is_windows:
    flags.append('-I' + str((prefix/'Library'/'include').abspath()))
else:
    flags.append('-I' + str((prefix/'include').abspath()))
    
if autowig.parser.plugin == 'libclang':
    kwargs['silent'] = True
    
asg = autowig.parser(asg, headers,
                          flags,
                          **kwargs)

Since most of **AutoWIG** guidelines are respected in this *C++* library, the `default` `controller` implementation is suitable.

In [ ]:
%%time

asg = autowig.controller(asg)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [ ]:
%%time

autowig.generator.plugin = 'boost_python_internal'  
wrappers = autowig.generator(asg,
                             module = os.path.join(GIT_ROOT, 'src', 'py', '_basic.cpp'),
                             decorator = os.path.join(GIT_ROOT, 'src', 'py', 'basic', '_basic.py'),
                             prefix = 'wrapper_')

The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [ ]:
%%time

wrappers.write()

Here is the list of the generated wrappers (untracked files).

In [ ]:
!git status

And here, we present the wrappers generated for the `BinomialDistribution` class.

In [ ]:
WRAPPER = os.path.join(GIT_ROOT, 'src', 'py', 'wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp')
!pygmentize {WRAPPER}

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.
To do so, we use available **Conda** recipes.

In [ ]:
subprocess.call('conda remove -y python-basic', shell=True)
CONDA_RECIPE = os.path.join(GIT_ROOT, 'bin', 'conda', 'python-basic')
subprocess.check_call('conda build --python=' + PYTHON_VERSION + ' ' + CONDA_RECIPE + ' -c statiskit -c defaults --override-channels',
                      shell=True)
subprocess.check_call('conda install -y python-basic --use-local -c statiskit -c defaults --override-channels',
                      shell=True)

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [ ]:
import basic
binomial = basic.BinomialDistribution(1, .5)
binomial

In [ ]:
binomial.pmf(0)

In [ ]:
binomial.pmf(1)

In [ ]:
binomial.n = 0
binomial

In [ ]:
binomial.pmf(0)

In [ ]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print(error)
else:
    raise Exception('A `basic.ProbabilityError` should have been raised')

Here is a report concerning objects wrapped using this notebook.

In [ ]:
import fp17
fp17.report(asg)